In [1]:
import pandas as pd
from pandas import DataFrame,Series
import numpy as np
import os
import datetime

#Plotting
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline

# sklearn stuff
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, mean_squared_error, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, Imputer 
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

import feature_pipelines as pipes

### Reading in Data

In [2]:
maindir = "/home/anerdi/Desktop/Zillow"
logerror = pd.read_csv(maindir + "/data/train_2016_v2.csv/train_2016_v2.csv")
logerror['weeknumber'] = logerror['transactiondate'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d').isocalendar()[1])
logerror['month'] = logerror['transactiondate'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d').month)
properties = pd.read_csv(maindir + "/data/properties_2016.csv/properties_2016.csv")

/home/anerdi/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#proportion of living area
properties['N-LivingAreaProp'] = properties['calculatedfinishedsquarefeet']/properties['lotsizesquarefeet']

#Ratio of the built structure value to land area
properties['N-ValueProp'] = properties['structuretaxvaluedollarcnt']/properties['landtaxvaluedollarcnt']

#Ratio of tax of property over parcel
properties['N-ValueRatio'] = properties['taxvaluedollarcnt']/properties['taxamount']

# Pool
properties['Pool'] = (properties['pooltypeid2'].fillna(0) + properties['pooltypeid7'].fillna(0)).astype(int)

In [4]:
# join on parcel id
data = pd.merge(properties,logerror[['parcelid','logerror','month']], on='parcelid')
del logerror

### New response variable 

In [5]:
data['overestimation'] = (data['logerror'] >= 0).astype(int)
data['extreme_overestimation'] = (data['logerror'] >= 1.5).astype(int)
data['extreme_underestimation'] = (data['logerror'] <= -1.5).astype(int)

### Feature Pipeline

In [6]:
# Setup variables considered in the model


# numerical variables
num_atts = ['bathroomcnt','bedroomcnt','buildingqualitytypeid','calculatedbathnbr','finishedfloor1squarefeet',
           'calculatedfinishedsquarefeet','finishedsquarefeet12','finishedsquarefeet13',
           'finishedsquarefeet15','finishedsquarefeet50','finishedsquarefeet6','fireplacecnt',
           'fullbathcnt','garagecarcnt','garagetotalsqft','latitude','longitude','lotsizesquarefeet',
           'poolcnt','poolsizesum','censustractandblock','roomcnt','threequarterbathnbr','unitcnt',
           'yardbuildingsqft17','yardbuildingsqft26','numberofstories',
            'structuretaxvaluedollarcnt','taxvaluedollarcnt','landtaxvaluedollarcnt','taxamount',
           'N-ValueRatio', 'N-LivingAreaProp', 'N-ValueProp']

# categorical varaibles
cat_atts = ['airconditioningtypeid','architecturalstyletypeid',
           'buildingclasstypeid','heatingorsystemtypeid','pooltypeid10','pooltypeid2',
            'pooltypeid7','propertylandusetypeid','regionidcounty',
           'storytypeid','typeconstructiontypeid','yearbuilt','fireplaceflag',
           'taxdelinquencyflag']

# Dictionary of categorical variables and their default levels
cat_dict = {key:value for key,value in {'airconditioningtypeid':[-1] + list(range(1,14)),
           'architecturalstyletypeid':[-1] + list(range(1,28)),
           'buildingclasstypeid':[-1] + list(range(1,6)),
            'heatingorsystemtypeid':[-1] + list(range(1,26)),
            'pooltypeid10': list(range(-1,2)),
            'pooltypeid2': list(range(-1,2)),
            'pooltypeid7': list(range(-1,2)),
            'Pool': [0,1],
            'propertylandusetypeid': [-1, 31,46,47,246,247,248,260,261,262,263,264,265,266,267,268,269,270,271,
                                     273,274,275,276,279,290,291],
            'regionidcounty': [2061,3101,1286],
            'storytypeid':[-1] + list(range(1,36)),
            'typeconstructiontypeid':[-1] + list(range(1,19)),
            'yearbuilt': [-1] + list(range(1885,2018)),
            'fireplaceflag': [-1] + ['True','False'],
            'taxdelinquencyflag': [-1] + ['Y','N']
           }.items() if key in cat_atts}

In [7]:
# Categorical pipeline
cat_pipeline = Pipeline([
        ('select_and_dummify', pipes.DF_Selector_GetDummies(cat_dict)),
    ])

# Numerical pipeline
num_pipeline = Pipeline([
        ('selector', pipes.DataFrameSelector(num_atts)),
        ('imputer', Imputer()),
        ('scaler', StandardScaler())
    ])

# Full pipeline
feature_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipeline", cat_pipeline)
    ])

### Splitting data into the 10-Folds

In [8]:
indices = np.arange(data.shape[0])

In [9]:
np.random.seed(19)
np.random.shuffle(indices) # in-place shuffling 
indices

array([ 2199, 86155, 84691, ..., 86952, 82677, 76398])

In [10]:
fold_indices = {(i+1):indices[i::10] for i in range(10)}

In [11]:
fold_indices

{1: array([ 2199, 83721, 29492, ..., 37852, 48911, 39220]),
 2: array([86155, 32252, 81949, ..., 57319, 13479, 33811]),
 3: array([84691, 37597,  3215, ..., 84821, 43372, 86952]),
 4: array([11172, 67082, 58364, ..., 74500, 63830, 82677]),
 5: array([78769, 73075, 17232, ..., 12489,   266, 76398]),
 6: array([53035, 17238, 32604, ..., 14649, 26827, 61025]),
 7: array([58194, 72307,  3380, ..., 57397, 68361, 53125]),
 8: array([66378, 81551, 66156, ..., 73922, 85799, 45218]),
 9: array([70318, 70507, 20646, ...,  7537, 69584, 17218]),
 10: array([42552, 66817, 57336, ..., 88913, 67815, 17738])}

## Layer 1 Classification - P(overestimation)

### Training Models on the 10 splits of data \ fold_i for i = 1,...,10 & obtaining level 1 data

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.base import clone

import warnings
warnings.filterwarnings("ignore")

In [13]:
feature_pipeline.fit(data)

FeatureUnion(n_jobs=1,
       transformer_list=[('num_pipeline', Pipeline(memory=None,
     steps=[('selector', DataFrameSelector(desired_cols=['bathroomcnt', 'bedroomcnt', 'buildingqualitytypeid', 'calculatedbathnbr', 'finishedfloor1squarefeet', 'calculatedfinishedsquarefeet', 'finishedsquarefeet12', 'finishedsquarefeet13', 'fi... 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]}))]))],
       transformer_weights=None)

In [14]:
models = [
    ("network", MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(3, 50), random_state=1)),
    ("logistic", LogisticRegression(penalty='l1', tol=0.01)),
    ("rf_1",RandomForestClassifier(max_depth=8, random_state=9,class_weight={0:0.52,1:0.48})),
    ("rf_2", RandomForestClassifier(max_depth=12, random_state=9)),
    ("rf_3", RandomForestClassifier(max_depth=12, random_state=9, class_weight={0:0.55,1:0.45})),
    ("xgb_1", XGBClassifier(max_depth=7, random_state=9, n_jobs=2)),
    ("xgb_2", XGBClassifier(max_depth=5, random_state=9, n_jobs=2)),
    ("lgbm_1", LGBMClassifier(random_state=9, n_jobs=2, n_estimators=100)),
    ("lgbm_2", LGBMClassifier(random_state=9, n_jobs=2, n_estimators=100, boosting_type = 'goss'))
]

In [15]:
level_one_data = data[['parcelid']].copy()

for pair in models:
    current_model_name,current_model = pair
    print("Current model: %s" % current_model_name)
    
    # initialize an NoneObject to be a placeholder for level-one data for current model
    model_preds = None 
    
    for fold_nbr in range(1,11):
        print("...working on fold %d" % fold_nbr)

        # set training data X \ fold
        current_traindata = data.iloc[np.setdiff1d(indices,fold_indices[fold_nbr]),]

        # get a clone of the model and fit the current training data
        print('......training model')
        clf = clone(current_model)
        clf.fit(feature_pipeline.transform(current_traindata), current_traindata['overestimation'])

        # level-one data (i.e., predict observations on current fold using reg)
        print('......obtaining level-one data')
        fold_data = data.iloc[fold_indices[fold_nbr]]
        fold_preds = Series(clf.predict_proba(feature_pipeline.transform(fold_data))[:,1], 
                            index=fold_indices[fold_nbr], name = current_model_name)

        # adding to the placeholder for level-one data
        if model_preds is not None:
            model_preds = pd.concat([model_preds, fold_preds])
        else:
            model_preds = fold_preds

        # some housecleaning
        del clf
    
    # add level-one predictions of current model to running dataframe
    level_one_data = pd.concat([level_one_data, model_preds], axis=1)
    print("")
    
print("all done!")

Current model: network
...working on fold 1
......training model
......obtaining level-one data
...working on fold 2
......training model
......obtaining level-one data
...working on fold 3
......training model
......obtaining level-one data
...working on fold 4
......training model
......obtaining level-one data
...working on fold 5
......training model
......obtaining level-one data
...working on fold 6
......training model
......obtaining level-one data
...working on fold 7
......training model
......obtaining level-one data
...working on fold 8
......training model
......obtaining level-one data
...working on fold 9
......training model
......obtaining level-one data
...working on fold 10
......training model
......obtaining level-one data

Current model: logistic
...working on fold 1
......training model
......obtaining level-one data
...working on fold 2
......training model
......obtaining level-one data
...working on fold 3
......training model
......obtaining level-one data
..

In [16]:
level_one_data.head()

,parcelid,network,logistic,rf_1,rf_2,rf_3,xgb_1,xgb_2,lgbm_1,lgbm_2
0,17073783,0.540814,0.586041,0.606695,0.725421,0.570743,0.703276,0.626319,0.600729,0.817952
1,17088994,0.575728,0.602346,0.520296,0.538926,0.501637,0.485501,0.552616,0.529095,0.461712
2,17100444,0.618680,0.516813,0.527499,0.537820,0.490139,0.591807,0.581336,0.555954,0.585355
3,17102429,0.667181,0.594042,0.535683,0.560426,0.524229,0.647888,0.661331,0.666883,0.656928
4,17109604,0.561772,0.588186,0.619031,0.568314,0.526031,0.475878,0.480612,0.461771,0.437258


In [17]:
data.shape[0]

90275

In [18]:
level_one_data.shape[0]==data.shape[0]

True

In [19]:
level_one_data.to_csv("/home/anerdi/Desktop/Zillow/levelonedata/stage1_l1data.csv.gz", compression ='gzip',
                      index=False)

### The Stacked Model

In [20]:
training_data = pd.merge(level_one_data, data[['overestimation','parcelid']], on='parcelid')
training_data.head()

,parcelid,network,logistic,rf_1,rf_2,rf_3,xgb_1,xgb_2,lgbm_1,lgbm_2,overestimation
0,17073783,0.540814,0.586041,0.606695,0.725421,0.570743,0.703276,0.626319,0.600729,0.817952,1
1,17088994,0.575728,0.602346,0.520296,0.538926,0.501637,0.485501,0.552616,0.529095,0.461712,1
2,17100444,0.618680,0.516813,0.527499,0.537820,0.490139,0.591807,0.581336,0.555954,0.585355,1
3,17102429,0.667181,0.594042,0.535683,0.560426,0.524229,0.647888,0.661331,0.666883,0.656928,0
4,17109604,0.561772,0.588186,0.619031,0.568314,0.526031,0.475878,0.480612,0.461771,0.437258,1


In [21]:
stacked = LogisticRegression(penalty='l1', tol=0.01)

In [22]:
stacked.fit(training_data[['network','rf_2','rf_3','xgb_1','xgb_2','lgbm_1','lgbm_2']], training_data['overestimation'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.01,
          verbose=0, warm_start=False)

In [23]:
stacked.coef_[0]

array([ 0.84417336, -0.1487704 , -0.3090904 ,  1.26490142,  0.6983782 ,
        0.94914558,  0.7787071 ])

In [24]:
stacked.intercept_

array([-2.04366505])

In [25]:
stacked.score(training_data[['network','rf_2','rf_3','xgb_1','xgb_2','lgbm_1','lgbm_2']], training_data['overestimation'])

0.58702928408099242

In [26]:
pred = stacked.predict(training_data[['network','rf_2','rf_3','xgb_1','xgb_2','lgbm_1','lgbm_2']])
true = training_data['overestimation']

In [27]:
print(recall_score(true,pred))
print(precision_score(true,pred))
print(f1_score(true,pred))

0.856824589421
0.590619988584
0.69924297885


In [28]:
confusion_matrix(true, pred, labels=[0,1])

array([[ 9683, 30123],
       [ 7262, 43459]])

### Training on full dataset

In [29]:
clf_ann = clone(models[0][1])
clf_rf2 = clone(models[3][1])
clf_rf3 = clone(models[4][1])
clf_xgb1 = clone(models[5][1])
clf_xgb2 = clone(models[6][1])
clf_lgbm1 = clone(models[7][1])
clf_lgbm2 = clone(models[8][1])

In [30]:
clf_ann.fit(feature_pipeline.transform(data), data['overestimation'])
clf_rf2.fit(feature_pipeline.transform(data), data['overestimation'])
clf_rf3.fit(feature_pipeline.transform(data), data['overestimation'])
clf_xgb1.fit(feature_pipeline.transform(data), data['overestimation'])
clf_xgb2.fit(feature_pipeline.transform(data), data['overestimation'])
clf_lgbm1.fit(feature_pipeline.transform(data), data['overestimation'])
clf_lgbm2.fit(feature_pipeline.transform(data), data['overestimation'])

LGBMClassifier(boosting_type='goss', colsample_bytree=1.0, learning_rate=0.1,
        max_bin=255, max_depth=-1, min_child_samples=10,
        min_child_weight=5, min_split_gain=0.0, n_estimators=100, n_jobs=2,
        num_leaves=31, objective=None, random_state=9, reg_alpha=0.0,
        reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=50000, subsample_freq=1)

In [31]:
models = [('ann',clf_ann),
          ('rf2',clf_rf2),
          ('rf3',clf_rf3),
          ('xgb1',clf_xgb1),
          ('xgb2',clf_xgb2),
          ('lgbm1',clf_lgbm1),
          ('lgbm2',clf_lgbm2)
         ]

In [32]:
overestimate_probs = pd.read_csv(maindir + "/data/properties_2016.csv/properties_2016.csv", usecols=['parcelid'])
for pair in models:
    model_name, model = pair
    probabilities = None
    for i in range(int(properties.shape[0] / 100000)):   
        # get current test features
        current_test_feats = feature_pipeline.transform(properties.iloc[i*100000:(i+1)*100000])

        # predict on current test obs
        current_probs = Series(model.predict_proba(current_test_feats)[:,1], name='%s_overestimate_prob'%model_name,
                              index = np.arange(i*100000,(i+1)*100000))

        if probabilities is not None:
            probabilities = pd.concat([probabilities, current_probs])
        else:
            probabilities = current_probs

    #  fencepost problem
    current_test_feats = feature_pipeline.transform(properties.iloc[2900000:])
    current_probs = Series(model.predict_proba(current_test_feats)[:,1], name='%s_overestimate_prob'%model_name,
                          index = np.arange(2900000,2985217))
    probabilities = pd.concat([probabilities, current_probs])
    overestimate_probs = pd.concat([overestimate_probs, probabilities], axis=1)

In [33]:
overestimate_probs['stacked_pred'] = 1 / (1 + np.exp(-stacked.intercept_[0]
         - overestimate_probs['ann_overestimate_prob']*stacked.coef_[0][0]
            - overestimate_probs['rf2_overestimate_prob']*stacked.coef_[0][1]
                - overestimate_probs['rf3_overestimate_prob']*stacked.coef_[0][2]
                     - overestimate_probs['xgb1_overestimate_prob']*stacked.coef_[0][3]
                        - overestimate_probs['xgb2_overestimate_prob']*stacked.coef_[0][4]
                            - overestimate_probs['lgbm1_overestimate_prob']*stacked.coef_[0][5]
                                - overestimate_probs['lgbm2_overestimate_prob']*stacked.coef_[0][6]))

In [34]:
overestimate_probs.head()

,parcelid,ann_overestimate_prob,rf2_overestimate_prob,rf3_overestimate_prob,xgb1_overestimate_prob,xgb2_overestimate_prob,lgbm1_overestimate_prob,lgbm2_overestimate_prob,stacked_pred
0,10754147,0.469632,0.466012,0.495383,0.658496,0.665265,0.631455,0.529722,0.608192
1,10759547,0.406392,0.555562,0.524675,0.536781,0.519523,0.552883,0.605128,0.523073
2,10843547,0.880731,0.548264,0.548752,0.430116,0.485831,0.575907,0.513572,0.569187
3,10859147,0.569588,0.663067,0.543329,0.679050,0.547760,0.614516,0.622471,0.617759
4,10879947,0.540791,0.519636,0.485341,0.502730,0.516258,0.486744,0.397359,0.488352


In [35]:
overestimate_probs.to_csv("/home/anerdi/Desktop/Zillow/twostagemodel/overestimate_probs_stacked_ann_rfs_xgbs_lgbms.csv.gz", 
                          index=False, compression="gzip")

## Layer 2 P(extreme_overestimation | overestimation) and P(extreme_underestimation | underestimation)

In [90]:
X_over = feature_pipeline.fit_transform(data[data['overestimation'] == 1])
y_extreme_over = data[data['overestimation'] == 1]['extreme_overestimation']
X_under = feature_pipeline.fit_transform(data[data['overestimation'] == 0])
y_extreme_under = data[data['overestimation'] == 0]['extreme_underestimation']

assert X_over.shape[0] == y_extreme_over.shape[0]
assert X_under.shape[0] == y_extreme_under.shape[0]

In [91]:
clf_rf_layer2_over = RandomForestClassifier(max_depth=15, random_state=9, class_weight='balanced')
clf_rf_layer2_over.fit(X_over, y_extreme_over)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=15, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=9,
            verbose=0, warm_start=False)

In [92]:
clf_rf_layer2_under = RandomForestClassifier(max_depth=12, random_state=9, class_weight='balanced')
clf_rf_layer2_under.fit(X_under, y_extreme_under)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=12, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=9,
            verbose=0, warm_start=False)

In [108]:
confusion_matrix(clf_rf_layer2_over.predict(X_over), y_extreme_over)

array([[50436,    33],
       [   70,    69]])

In [110]:
confusion_matrix(clf_rf_layer2_under.predict(X_under), y_extreme_under)

array([[39520,    17],
       [   88,    42]])

In [93]:
models = [('over',clf_rf_layer2_over),('under',clf_rf_layer2_under)]
extreme_probs = pd.read_csv(maindir + "/data/properties_2016.csv/properties_2016.csv", usecols=['parcelid'])
for pair in models:
    model_name, model = pair
    probabilities = None
    for i in range(int(properties.shape[0] / 100000)):   
        # get current test features
        current_test_feats = feature_pipeline.transform(properties.iloc[i*100000:(i+1)*100000])

        # predict on current test obs
        current_probs = DataFrame(model.predict_proba(current_test_feats), name='extreme_%s_prob'%model_name,
                              index = np.arange(i*100000,(i+1)*100000))

        if probabilities is not None:
            probabilities = pd.concat([probabilities, current_probs])
        else:
            probabilities = current_probs

    #  fencepost problem
    current_test_feats = feature_pipeline.transform(properties.iloc[2900000:])
    current_probs = Series(model.predict_proba(current_test_feats)[:,1], name='extreme_%s_prob'%model_name,
                          index = np.arange(2900000,2985217))
    probabilities = pd.concat([probabilities, current_probs])
    extreme_probs = pd.concat([extreme_probs, probabilities], axis=1)

In [95]:
extreme_probs.head()

,parcelid,extreme_over_prob,extreme_under_prob
0,10754147,0.544759,0.192589
1,10759547,0.283555,0.220792
2,10843547,0.357077,0.124701
3,10859147,0.184405,0.136827
4,10879947,0.238265,0.059137


## Overall probs 

In [97]:
assert (overestimate_probs['parcelid'] == extreme_probs['parcelid']).all() == True

In [98]:
overall_probs = pd.merge(overestimate_probs[['parcelid','stacked_pred']], extreme_probs, on='parcelid')

In [104]:
overall_probs['over_but_not_extreme'] = overall_probs['stacked_pred']*(1 - overall_probs['extreme_over_prob'])
overall_probs['over_and_extreme'] = overall_probs['stacked_pred']*overall_probs['extreme_over_prob']
overall_probs['under_but_not_extreme'] = (1 - overall_probs['stacked_pred'])*(1 - overall_probs['extreme_under_prob'])
overall_probs['under_and_extreme'] = (1 - overall_probs['stacked_pred'])*overall_probs['extreme_under_prob']

In [105]:
overall_probs.head()

,parcelid,stacked_pred,extreme_over_prob,extreme_under_prob,over_but_not_extreme,over_and_extreme,under_but_not_extreme,under_and_extreme
0,10754147,0.473844,0.544759,0.192589,0.215713,0.258131,0.424824,0.101332
1,10759547,0.474768,0.283555,0.220792,0.340146,0.134623,0.409265,0.115967
2,10843547,0.732197,0.357077,0.124701,0.470746,0.261451,0.234407,0.033395
3,10859147,0.606960,0.184405,0.136827,0.495034,0.111926,0.339262,0.053778
4,10879947,0.529348,0.238265,0.059137,0.403223,0.126125,0.442819,0.027833


In [107]:
overall_probs[['over_but_not_extreme','over_and_extreme','under_but_not_extreme','under_and_extreme']].sum(axis=1).sum()

2985217.0

In [111]:
overall_probs.to_csv("/home/anerdi/Desktop/Zillow/twostagemodel/two_layer_probabilities.csv.gz", 
                          index=False, compression="gzip")